<a href="https://colab.research.google.com/github/audrism/2023Hackathon/blob/main/MP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MP5

# Practice using text embeddings for search and for classification

## store in a vector database

## find nearest neighbors

## classify by repo typy and by repo

## To use upload/download you need to login into HGF CLI; use write token created on HGF

In [1]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `w` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface

In [2]:
!huggingface-cli download --repo-type dataset --local-dir . fdac24/MP4 --include "*.json.gz"
!huggingface-cli download --repo-type dataset --local-dir . fdac24/MP3 --include "*.json.gz"

Fetching 38 files:   0% 0/38 [00:00<?, ?it/s]Downloading 'amarlow6_embed.json.gz' to '.cache/huggingface/download/amarlow6_embed.json.gz.25ac1cfef8e377b730e4b7f4f9917789018e3a8f82f2dc002cfb5e30a5728091.incomplete'

amarlow6_embed.json.gz:   0% 0.00/9.32M [00:00<?, ?B/s]

audris_embed.json.gz:   0% 0.00/108k [00:00<?, ?B/s]


amuell11_embed.json.gz:   0% 0.00/1.23M [00:00<?, ?B/s]



audris_embed.json.gz: 100% 108k/108k [00:00<00:00, 5.05MB/s]
Download complete. Moving file to audris_embed.json.gz


ckornega_embed.json.gz:   0% 0.00/18.0M [00:00<?, ?B/s]




cstefani_embed.json.gz:   0% 0.00/19.6M [00:00<?, ?B/s]





ccanonac_embed.json.gz:   0% 0.00/1.30M [00:00<?, ?B/s]






aking100_embed.json.gz:   0% 0.00/18.5M [00:00<?, ?B/s]


ccanonac_embed.json.gz: 100% 1.30M/1.30M [00:00<00:00, 15.1MB/s]
Download complete. Moving file to ccanonac_embed.json.gz
amuell11_embed.json.gz: 100% 1.23M/1.23M [00:00<00:00, 10.4MB/s]
Download complete. Moving file to amuell11_embed.json.gz
bmilstea_em

# Now read in embeddings

In [3]:
import json, re, gzip
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json, re, gzip

utids4 = !ls *_embed.json.gz|sed 's|_embed.json.gz||'
labels = []
X = []
for utid in (utids4):
  #print (utid)
  fi = gzip.open(f"{utid}_embed.json.gz", 'r')
  for line in fi:
   line = line.strip ()
   #(line)
   if len(line) < 10: next
   id, typ, section, vector = json.loads(line)
   labels.append({'id':id, 'type':typ, 'section':section, 'utid':utid, 'emb':np.array(vector)})

# LOOK FOR ANOMALIES
check = pd.crosstab([l['utid'] for l in labels], [l['section'] == '0: Preamble' for l in labels])
print(check)




col_0     False  True 
row_0                 
aking100   2729    202
amarlow6   1280    106
amuell11      0    173
audris       13      2
bmilstea    394     48
ccanonac      0    182
ckornega   2581    194
cstefani   2746    188
cvy221        0    192
ddelrosa   2643    176
dhodge12   2551    189
dpate139   2631    185
ezhao1     2894    183
fchernow      0    200
gpatel8    2870    199
ibhandar   2990    191
jaugust4      0    175
jclar166      0    191
jkutbay       0    174
jtayl219      0    178
knguye34   2495    194
lscott32   2609    189
mherna21      0    192
monim      2789    179
ncoffey3      0    191
pkx959     2477    177
rking61    2968    184
sbandar1   2574    176
sgray38     146     15
snidiff1   2524    194
tcatunca      0    198
vbroda     2580    120
wduff         0    197
ygaikwad      0    179
yhg461        0    181


# instances with a single section are problematic: exclude

In [4]:
df = pd.DataFrame(labels)
utidsG = []
dfG = pd.DataFrame()
Xg = []
for utid in (utids4):
  df1 = df[(df['utid']==utid) & (df['section'] != '0: Preamble')]
  if len(df1) > 0:
    utidsG.append(utid)
    dfG = pd.concat([dfG, df[df['utid']==utid] ], ignore_index=True)


# Now use embedding to find most similar sections

In [6]:
# this takes forever, use vector database instead
def csim(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = sum(a * a for a in v1) ** 0.5
    magnitude_v2 = sum(b * b for b in v2) ** 0.5
    return dot_product / (magnitude_v1 * magnitude_v2)

def mostSim (i):
  mx = -1
  mxi = -1
  for j in range(len(labels)):
    if(i != j):
      v = csim(dfG.iloc[i]['emb'], dfG.iloc[j]['emb'])
      if v > mx:
        mx = v
        mxi = j
  return (mxi,mx,dfG[mxi])

closest = []
for i in range(len(labels)):
  closest.append(mostSim(i))


KeyboardInterrupt: 

## install vector database

In [7]:
! pip install pymilvus milvus --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00


In [8]:
from milvus import default_server
from pymilvus import connections, utility
default_server.start()
connections.connect(host="127.0.0.1", port=default_server.listen_port)

In [29]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

COLLECTION_NAME = "hfh"
DIMENSION = 768
BATCH_SIZE = 128
TOPK = 3
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="repo", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="section", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="utid", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)
collection = Collection(name=COLLECTION_NAME, schema=schema)

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [30]:
for i in range(len(dfG)):
   d = dfG.iloc[i]
   #df1 = df[(df['ID']== id) & (df['section'] == section)]['content']
   #if (len(df1) > 1):    df1 = df1.iloc[0]
   #else:    if (len(df1) < 10): continue
   #cnt = df1
   section = d['section']
   lenT = len(section)
   if lenT > 30: section = section[:30]
   ins =  [ [ d['id'] ], [ section ], [ d['type'] ], [ d['utid'] ], [ d['emb' ] ] ]
   collection.insert(ins)

In [34]:
neighbors = []
for i in range(len(dfG)):
  v = collection.search (
    data= [ dfG.iloc[i]['emb'] ],  # Embeded search value
    anns_field="embedding",        # Search across embeddings
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=TOPK,  # Return 3 nearest neighbours
    output_fields=[ "repo", "type", "section", 'utid' ])
  neighbors .append(v)


# See if the closest neighor has the same repo, utid, or type

In [40]:

tot = 0
mmatch = 0
for i in range(len(dfG)):
  if dfG.iloc[i]['id'] ==neighbors[i][0][0].entity.get("repo"):
      mmatch+=1
  tot += 1
print(mmatch, tot, mmatch/tot)
umatch = 0
for i in range(len(dfG)):
  if dfG.iloc[i]['utid'] ==neighbors[i][0][0].entity.get("utid"):
      umatch+=1
print(umatch, tot, umatch/tot)
tmatch = 0
for i in range(len(dfG)):
  if dfG.iloc[i]['type'] ==neighbors[i][0][0].entity.get("type"):
      tmatch+=1
print(tmatch, tot, umatch/tot)

11758 50775 0.23157065484982767
13295 50775 0.2618414574101428
37128 50775 0.2618414574101428


# Create a classifier based on embeddings

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Regression needs separate columns
X = np.array(dfG['emb'])
Xh = np.vstack(X)

rtype = [ (l=='model')+1 for l in dfG['type']]
# try type
X_train, X_test, y_train, y_test = train_test_split(Xh, rtype, test_size=0.2)
#print(X_train)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
print("Accuracy for type:", accuracy)

Accuracy for type: 0.9136386016740522


In [78]:
classifier.fit(X_train[:,:10], y_train)
accuracy = classifier.score(X_test[:,:10], y_test)
print("Accuracy for type (10 predictors):", accuracy)

Accuracy for type (10 predictors): 0.7760709010339734


In [79]:
from sklearn.svm import SVC
clsvm = SVC(kernel='linear')
clsvm = clsvm.fit(X_train[:,:10], y_train)
accuracy = clsvm.score(X_test[:,:10], y_test)
print("Accuracy for type (svm, 10 predictors):", accuracy)

Accuracy for type (svm,10 predictors): 0.7917282127031019


In [80]:
# try repo
rrepo = dfG['id']
X_train, X_test, y_train, y_test = train_test_split(Xh, rrepo, test_size=0.4)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
print("Accuracy for repo:", accuracy)

Accuracy for repo: 0.016642048252092565


In [81]:
clsvm = SVC(kernel='linear')
clsvm = clsvm.fit(X_train[:,:10], y_train)
accuracy = clsvm.score(X_test[:,:10], y_test)
print("Accuracy for repo (svm, 10 predictors):", accuracy)

Accuracy for repo (svm, 10 predictors): 0.010881339241752832
